# Exam 2 (100 points total)

As described in the course's Honor Code policy, you may use your notes, and anything provided as part of this course.  You may use your computed to access this allowed information, to use tools to answer these questions (i.e., Jupyter, word processing software, drawing software, and browser to access allowed information).  You may not collaborate with anyone else, and you may not access any websites not associated with the course, except for web-based drawing programs.

Complete and submit this notebook.  In any problem asking for a query, you may decompose the problem into a simpler query that uses views or temporary tables.

In [1]:
%load_ext sql
%sql sqlite:///

u'Connected: None@'

## Problem 1 (20 points total)

### Part a (10 points)

Give one advantage of normalizing data.

* It reduces data redundancy, thus reducing the various kinds of data anomalies.
* It is a way of reducing the costs of checking for and maintaining data consistency.
* It eliminates redundant data, possibly reducing the amount of storage necessary.
* It eliminates redundant data, possibly reducing the amount of data per table, thus any query on just that table will be faster.
* If breaks tables into smaller relations.  When the data from only one relation is needed, it should be faster because there is less data to read and use.

### Part b (10 points)

Give two reasons why normalizing data can make queries slower.  Or equivalently, why denormalizing data can make queries faster.

* Data split into separate tables by normalization needs to be joined, and joining is expensive, so accessing data is slower than if it were in a single denormalized table.
* Data split into separate tables by normalization needs to be joined, and the resulting joined table isn't indexed, so accessing data is slower than if it were in a single denormalized table with an appropriate index.
* Data in denormalized tables has better locality because each single table is typically stored contiguously.
* Denormalized tables can contain pre-computed derived values so that these values don't need to be computed in queries.

## Problem 2 (60 points total)

Let's first define a couple tables and some sample data.  We have pilots, who can fly various kinds of airplanes, and we have a hangar containing some airplanes.  Our pilots might be able to fly airplanes that are not currently available in the hangar, so we do *not* have a foreign key relationship between the tables.

In [2]:
%%sql

DROP TABLE IF EXISTS PilotSkills;
DROP TABLE IF EXISTS Hangar;

CREATE TABLE Hangar (
    plane_name CHAR(15) NOT NULL,
    PRIMARY KEY (plane_name)
);

CREATE TABLE PilotSkills (
    pilot_name CHAR(15) NOT NULL,
    plane_name CHAR(15) NOT NULL,
    PRIMARY KEY (pilot_name, plane_name)
);

INSERT INTO Hangar VALUES ('B-1 Bomber');
INSERT INTO Hangar VALUES ('B-52 Bomber');
INSERT INTO Hangar VALUES ('F-14 Fighter');

INSERT INTO PilotSkills VALUES ('Celko',    'Piper Cub');
INSERT INTO PilotSkills VALUES ('Higgins',  'B-52 Bomber');
INSERT INTO PilotSkills VALUES ('Higgins',  'F-14 Fighter');
INSERT INTO PilotSkills VALUES ('Higgins',  'Piper Cub');
INSERT INTO PilotSkills VALUES ('Jones',    'B-52 Bomber');
INSERT INTO PilotSkills VALUES ('Jones',    'F-14 Fighter');
INSERT INTO PilotSkills VALUES ('Smith',    'B-1 Bomber');
INSERT INTO PilotSkills VALUES ('Smith',    'B-52 Bomber');
INSERT INTO PilotSkills VALUES ('Smith',    'F-14 Fighter');
INSERT INTO PilotSkills VALUES ('Wilson',   'B-1 Bomber');
INSERT INTO PilotSkills VALUES ('Wilson',   'B-52 Bomber');
INSERT INTO PilotSkills VALUES ('Wilson',   'F-14 Fighter');
INSERT INTO PilotSkills VALUES ('Wilson',   'F-17 Fighter');

Done.
Done.
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [24]:
%%sql
SELECT pilot_name,CASE WHEN plane_name = 'B-52 Bomber' THEN 1 ELSE 2 END AS ability 
FROM PilotSkills


Done.


pilot_name,ability
Celko,2
Higgins,1
Higgins,2
Higgins,2
Jones,1
Jones,2
Smith,2
Smith,1
Smith,2
Wilson,2


### Which pilots can fly all of the available airplanes?

This is the question we want to ask.  While this question is simple and straight-forward, we'll see that it leaves some room for ambiguity.

This is an example of what is sometimes termed *relational division* because it roughly the inverse of the cartesian product.  Here, PilotSkills &divide; Hangar &asymp; Answer.  

### Part a (20 points)

Write a query **using quantification** that answers the question.  With the given data, the resulting pilots should be Smith and Wilson.  Each of those pilots can fly the three kinds of airplanes in the hangar.

In [27]:
%%sql
SELECT DISTINCT pilot_name
FROM PilotSkills AS PS1
WHERE NOT EXISTS
      (SELECT *
       FROM Hangar
       WHERE NOT exists
             (SELECT *
              FROM PilotSkills AS PS2
              WHERE (PS1.pilot_name = PS2.pilot_name) AND
                    (PS2.plane_name = Hangar.plane_name)));


Done.


pilot_name
Smith
Wilson


In [22]:
%%sql
SELECT *
       FROM Hangar
       WHERE not EXISTS
             (SELECT *
              FROM PilotSkills AS PS2
              WHERE 
                    (PS2.plane_name = Hangar.plane_name));

Done.


plane_name


### Part b (20 points)

Write a query **using joining, grouping, and aggregation** to answer the question.  Again, with the given data, the resulting pilots should be Smith and Wilson.

In [32]:
%%sql
SELECT PS.pilot_name
FROM PilotSkills AS PS
INNER JOIN Hangar AS H ON PS.plane_name = H.plane_name
GROUP BY PS.pilot_name
HAVING COUNT(PS.plane_name) = (SELECT COUNT(plane_name) FROM Hangar);

Done.


pilot_name
Higgins
Jones
Smith
Wilson


### Aside

While **not a requirement for the previous parts**, the most straightforward answers to the previous two parts will slightly differ semantically.  Consider if we had no planes available.  You can try this by commenting out the Hangar data (remember:  add two dashes at the beginning of each line).

The first query will likely result in all pilots.  After all, every one of the pilots is able to fly at least the zero available planes (in fact, so are you!).  Going back to the division analogy, this is like starting with all the pilots and planes, then dividing by zero planes and getting infinite (all) the pilots.

The second query will likely result in zero pilots.  This doesn't really correspond to the division analogy.

### Part c (20 points)

Write a query **using joining, grouping, and aggregation** to answer the question.

However, this time, we want to return to our division analogy and have no "remainder".  I.e., we want only the pilots who are able to fly *exactly* the set of available planes.  With the given data, the resulting pilots should be only Smith.  After all, Wilson can fly more than just the available planes.

As a hint, you'll want to use an **outer join** to create a NULL whenever the pilot's skills correspond to an unavailable plane.

In [5]:
%%sql
SELECT PS.pilot_name
FROM PilotSkills AS PS
LEFT OUTER JOIN Hangar AS H ON PS.plane_name = H.plane_name
GROUP BY PS.pilot_name
HAVING COUNT(PS.plane_name) = (SELECT COUNT(plane_name) FROM Hangar) AND
       COUNT(H.plane_name) = (SELECT COUNT(plane_name) FROM Hangar);

Done.


pilot_name,count()
Celko,1
Higgins,3
Jones,2
Smith,3
Wilson,4


## Problem 3 (20 points)

We have three tables with the following schemas:
* OuterTable(entity_id, base_id, ...)
* InnerTable(base_id, perf_id, ...)
* HugeTable(entity_id, perf_id, ...)

The ellipses indicate that there may be other fields irrelevant to the problem.

You are given the following cursor-based code.  (This won't run in SQLite because we haven't defined the tables, and SQLite only supports cursors in the application language.)

In [6]:
%%sql
DECLARE @EntityId INT;
DECLARE @PerfId INT;
DECLARE @BaseId INT;
DECLARE @UpdateStatus INT;

DECLARE outer_cursor CURSOR FOR
SELECT entity_id, base_id
FROM OuterTable;

OPEN outerCursor
FETCH NEXT FROM outer_cursor INTO @EntityId, @BaseId;

WHILE @@FETCH_STATUS = 0
BEGIN
    DECLARE inner_cursor CURSOR FOR
    SELECT perf_id
    FROM InnerTable
    WHERE base_id = @BaseId;

    OPEN innerCursor;
    FETCH NEXT FROM inner_cursor INTO @PerfId;
    SET @UpdateStatus = @@FETCH_STATUS;
    WHILE @UpdateStatus = 0
    BEGIN
        UPDATE HugeTable
        SET entity_id = @EntityId
        WHERE perf_id = @PerfId;

        FETCH NEXT FROM inner_cursor INTO @PerfId;
        SET @UpdateStatus = @@FETCH_STATUS
    END       
    CLOSE innerCursor;
    DEALLOCATE innerCursor;

    FETCH NEXT FROM outerCursor INTO @EntityId, @BaseId;
END

CLOSE outerCursor;
DEALLOCATE outerCursor;

(sqlite3.OperationalError) near "DECLARE": syntax error [SQL: u'DECLARE @EntityId INT;']


The previous code must have been written by a SQL newbie.  It is overly long and slow because it unnecessarily uses cursors.

Write equivalent SQL code that doesn't use cursors.  (Again, you won't be able to test it here, since we haven't defined the tables and data.)

In [7]:
%%sql

--Here is some test data.
DROP TABLE IF EXISTS OuterTable;
DROP TABLE IF EXISTS InnerTable;
DROP TABLE IF EXISTS HugeTable;

CREATE TABLE OuterTable (entity_id INT, base_id INT);
CREATE TABLE InnerTable (base_id INT, perf_id INT);
CREATE TABLE HugeTable  (entity_id INT, perf_id INT);

INSERT INTO OuterTable VALUES (1001, 11);
INSERT INTO OuterTable VALUES (2002, 22);
INSERT INTO OuterTable VALUES (4004, 44);

INSERT INTO InnerTable VALUES (11, 101);
INSERT INTO InnerTable VALUES (22, 102);
INSERT INTO InnerTable VALUES (33, 303);

INSERT INTO HugeTable VALUES (1, 101);
INSERT INTO HugeTable VALUES (2, 101);
INSERT INTO HugeTable VALUES (3, 303);
INSERT INTO HugeTable VALUES (4, 404);

SELECT * FROM HugeTable;

Done.
Done.
Done.
Done.
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
Done.


entity_id,perf_id
1,101
2,101
3,303
4,404


In [8]:
%%sql

--Adapted from:
--https://blogs.msdn.microsoft.com/sqlprogrammability/2008/03/18/increase-your-sql-server-performance-by-replacing-cursors-with-set-operations/

--The answer
DROP VIEW IF EXISTS Changes;

CREATE VIEW Changes AS
SELECT i.perf_id, o.entity_id
FROM InnerTable i
INNER JOIN OuterTable o on i.base_id = o.base_id;

UPDATE HugeTable
SET entity_id = (SELECT Changes.entity_id
                 FROM Changes
                 WHERE HugeTable.perf_id = Changes.perf_id)
WHERE EXISTS (SELECT Changes.entity_id
              FROM Changes
              WHERE HugeTable.perf_id = Changes.perf_id);

--A version using the simpler UPDATE FROM:
--UPDATE HugeTable
--SET h.entity_id = c.entity_id
--FROM HugeTable h
--INNER JOIN Changes c on h.perf_id = c.perf_id;


--Test:
SELECT * FROM HugeTable;

Done.
Done.
2 rows affected.
Done.


entity_id,perf_id
1001,101
1001,101
3,303
4,404
